# Laboratorium nr 4

## Konfiguracja


Poniższe narzędzie zostało zmodyfikowane, tak aby można było z niego korzystać w nowszych IDE, które nie wspierają `%matplotlib notebook`.
Aby korzystać z `%matplotlib widget` należy doinstalować pakiet `ipympl` np. za pomocą polecenia `pip install ipympl`. Jeżeli po zainstalowaniu będą występować błędy, należy korzystać z backendu `notebook`.

Wykorzystywane biblioteki:
- matplotlib
- numpy
- ipympl
- sortedcontainers

Nowe funkcje:
- Dodanie do konstruktora klasy Scene argumentu `title`, który pozwala na ustawienie tytułu danej sceny
- Dodanie do konstruktora klasy Plot argumentu `title`, który pozwala na ustawienie tytułu wykresu / okna widżetu
- Dodanie możliwości definiowania własnych zakresów osi OX oraz OY dla wykresów (argumenty klasy Plot: `xlim` oraz `ylim` jako krotka dwuelementowa definiująca przedział, gdy zostanie podany tylko zakres dla jednej osi, druga oś będzie automatycznie skalowana)

Naprawione bugi:
- Naprawiono problem, gdy po utworzeniu nowego pustego wykresu, tworzyła się kopia poprzednio utworzonego (kopiowały się sceny z ostatniego obiektu plot)
- Poprawiono metodę zamykania figur (tworzony jest okrąg o środku pierwszego punktu i promieniu adekwatnym do zakresów osi)
- Nowo dodane figury, punkty, linie są zapisywane osobno dla każdej sceny
- Zmieniono wyświetlanie się punktów i linii (punkty wyświetlają się przed liniami)



In [ ]:
# Wybór backendu matplotlib
# %matplotlib notebook
%matplotlib widget

# Narzędzie jest oparte o kilka zewnętrznych bibliotek, które potrzebujemy najpierw zaimportować.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

# Parametr określający jak blisko (w odsetku całego widocznego zakresu) punktu początkowego 
# wielokąta musimy kliknąć, aby go zamknąć.
TOLERANCE = 0.02

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

# Klasa ta trzyma obecny stan wykresu oraz posiada metody, które mają zostać wykonane
# po naciśnięciu przycisków.
class _Button_callback(object):
    def __init__(self, scenes, xlim=[], ylim=[]):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = [[] for _ in range(len(scenes))]
        self.adding_lines = False
        self.added_lines = [[] for _ in range(len(scenes))]
        self.adding_rects = False
        self.added_rects = [[] for _ in range(len(scenes))]
        # DODANO: zapisz tytuł sceny
        self.scene_title = self.scenes[0].title if len(self.scenes) and self.scenes[0].title else ""
        # DODANO: ustawienie zakresów osi wykresu
        self.xlim = xlim
        self.ylim = ylim
        self.custom_axis = len(xlim) != 0 or len(ylim) != 0

    def set_axes(self, ax):
        self.ax = ax
        
    # Metoda ta obsługuje logikę przejścia do następnej sceny.
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        # ADDED: Change scene title
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = not self.custom_axis)

    # Metoda ta obsługuje logikę powrotu do poprzedniej sceny.
    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        # ADDED: Change scene title
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = not self.custom_axis)
        
    # Metoda ta aktywuje funkcję rysowania punktów wyłączając równocześnie rysowanie 
    # odcinków i wielokątów.
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points[self.i].append(PointsCollection([]))
            
    # Metoda ta aktywuje funkcję rysowania odcinków wyłączając równocześnie
    # rysowanie punktów i wielokątów.     
    def add_line(self, event):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines[self.i].append(LinesCollection([]))

    # Metoda ta aktywuje funkcję rysowania wielokątów wyłączając równocześnie
    # rysowanie punktów i odcinków.
    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects[self.i].append(LinesCollection([]))
        self.rect_points = []
        
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        return self.added_points[self.i]
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        return self.added_lines[self.i]
        
    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        return self.added_rects[self.i]
  
    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        return Scene(self.added_points[self.i], self.added_lines[self.i]+self.added_rects[self.i])


    # Metoda odpowiedzialna za właściwą logikę rysowania nowych elementów. W
    # zależności od włączonego trybu dodaje nowe punkty, początek, koniec odcinka
    # lub poszczególne wierzchołki wielokąta. Istnieje ciekawa logika sprawdzania
    # czy dany punkt jest domykający dla danego wielokąta. Polega ona na tym, że
    # sprawdzamy czy odległość nowego punktu od początkowego jest większa od
    # średniej długości zakresu pomnożonej razy parametr TOLERANCE.   
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[self.i][-1].add_points([new_point])
            self.draw(autoscaling = False,hand_drawing=True)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[self.i][-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False,hand_drawing=True)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[self.i][-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False,hand_drawing=True)
            elif len(self.rect_points) > 1:
                # DODANO: zmiana metody wykrywania zakończenia figury.
                # Teraz punkt self.rect_points[0] jest środkiem okręgu o promieniu TOLERANCE*(mediana zakresów wykresu)
                # Gdy użytkownik kliknie w obrębie tego okręgu, figura zostanie zamknięta
                if (new_point[0] - self.rect_points[0][0]) ** 2 + (new_point[1] - self.rect_points[0][1]) ** 2 <= (np.mean([self.ax.get_xlim()[1] - self.ax.get_xlim()[0], self.ax.get_ylim()[1]-self.ax.get_ylim()[0]])*TOLERANCE) ** 2:
                    self.added_rects[self.i][-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:    
                    self.added_rects[self.i][-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False, hand_drawing=True)
    
    # Metoda odpowiedzialna za narysowanie całego wykresu. Warto zauważyć,
    # że zaczyna się ona od wyczyszczenia jego wcześniejszego stanu. Istnieje w
    # niej nietrywialna logika zarządzania zakresem wykresu, tak żeby, w zależności
    # od ustawionego parametru autoscaling, uniknąć sytuacji, kiedy dodawanie
    # nowych punktów przy brzegu obecnie widzianego zakresu powoduje niekorzystne
    # przeskalowanie.
    def draw(self, autoscaling = True, hand_drawing = False):
        # DODANO: ustawienie zakresów osi wykresu
        autoscale_axis='both'
        if not autoscaling:
            if self.custom_axis:
                xlim = self.xlim  
                ylim = self.ylim  

            choose_autoscale_axis = [False, False] # [OX, OY]

            if not self.custom_axis or not self.xlim:
                xlim = self.ax.get_xlim()
                choose_autoscale_axis[0] = True
            
            if not self.custom_axis or not self.ylim:
                ylim = self.ax.get_ylim()
                choose_autoscale_axis[1] = True

            if choose_autoscale_axis == [True, False]:
                autoscale_axis = 'x'
                autoscaling = not hand_drawing
            if choose_autoscale_axis == [False, True]:
                autoscale_axis = 'y'
                autoscaling = not hand_drawing

        self.ax.clear()
        # TODO: dodawanie punktów do scen zamiast do jednego kontenera (tyle added_points ile scen :)
        for collection in (self.scenes[self.i].points + self.added_points[self.i]):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs, zorder=3)
        for collection in (self.scenes[self.i].lines + self.added_lines[self.i] + self.added_rects[self.i]):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(enable=autoscaling, axis=autoscale_axis)

        if autoscale_axis == 'y':
            self.ax.set_xlim(xlim)

        if autoscale_axis == 'x':
            self.ax.set_ylim(ylim)
      
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
            
        # DODANO: tytuł sceny
        fig = plt.gcf() # Pobierz aktywne okno
        fig.suptitle(self.scene_title)
        
        plt.draw()


### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [ ]:
# Klasa Scene odpowiada za przechowywanie elementów, które mają być
# wyświetlane równocześnie. Konkretnie jest to lista PointsCollection i
# LinesCollection.
class Scene:
    def __init__(self, points=[], lines=[], title=""):
        self.points=points
        self.lines=lines
        # ADDED: Scene title
        self.title = '\n'+title if title != "" else ""

# Klasa PointsCollection gromadzi w sobie punkty jednego typu, a więc takie,
# które zostaną narysowane w takim samym kolorze i stylu. W konstruktorze
# przyjmuje listę punktów rozumianych jako pary współrzędnych (x, y). Parametr
# kwargs jest przekazywany do wywołania funkcji z biblioteki MatPlotLib przez
# co użytkownik może podawać wszystkie parametry tam zaproponowane.        
class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points = self.points + points

# Klasa LinesCollection podobnie jak jej punktowy odpowiednik gromadzi
# odcinki tego samego typu. Tworząc ją należy podać listę linii, gdzie każda
# z nich jest dwuelementową listą punktów – par (x, y). Parametr kwargs jest
# przekazywany do wywołania funkcji z biblioteki MatPlotLib przez co użytkownik
# może podawać wszystkie parametry tam zaproponowane.
class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

# Klasa Plot jest najważniejszą klasą w całym programie, ponieważ agreguje
# wszystkie przygotowane sceny, odpowiada za stworzenie wykresu i przechowuje
# referencje na przyciski, dzięki czemu nie będą one skasowane podczas tzw.
# garbage collectingu.
class Plot:
    def __init__(self, scenes = [], points = [], lines = [], json = None, title="", xlim=[], ylim=[]):
        # DODANO: tytuł wykresu
        self.plot_title = title
        # DODANO: ustawienie zakresów osi wykresu
        self.xlim = xlim
        self.ylim = ylim
        self.is_not_custom_axis = len(xlim)==0 and len(ylim)==0

        # NAPRAWIONO: pobieranie scen z wcześniej utworzonego wykresu
        if not scenes:
            self.scenes = [Scene()]
        else:
            self.scenes = scenes

        if json is None:
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
    
    # Ta metoda ma szczególne znaczenie, ponieważ konfiguruje przyciski i
    # wykonuje tym samym dość skomplikowaną logikę. Zauważmy, że konfigurując każdy
    # przycisk podajemy referencję na metodę obiektu _Button_callback, która
    # zostanie wykonana w momencie naciśnięcia.
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    # Metoda toJson() odpowiada za zapisanie stanu obiektu do ciągu znaków w
    # formacie JSON.
    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])    
    
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        if self.callback:
            result = self.callback.get_added_points()
            return result
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        if self.callback:
            result = self.callback.get_added_lines()
            return result

    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        if self.callback:
            result = self.callback.get_added_figure()
            return result

    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        if self.callback:
            result = self.callback.get_added_elements()
            return result

    # Główna metoda inicjalizująca wyświetlanie wykresu.
    def draw(self):
        plt.close()
        # DODANO: tytuł wykresu
        fig = plt.figure(num=self.plot_title)
        # DODANO: ustawienie zakresów osi wykresu
        self.callback = _Button_callback(self.scenes, self.xlim, self.ylim)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        # DODANO: ustawienie zakresów osi wykresu
        self.callback.draw(self.is_not_custom_axis)
        

### Przykłady użycia

##### Proste rysowanie

Rysowanie prostego rysunku złożonego ze statycznych punktów i odcinków jest stosunkowo proste. Należy utworzyć zmienną z obiektem `Plot`, który przyjmuje w konstruktorze listę obiektów `PointsCollection` jako parametr `points` oraz listę `LinesCollection` jako parametr `lines`. Następnie należy wywołać jego metodę `draw()`.

```python
plot = Plot(points=[PointsCollection([(1, 2), (3, 1.5), (2, -1)]),
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")],
            lines=[LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])])
plot.draw()     
```

##### Rysunek dynamiczny

To narzędzie pozwala na wizualizację "rysunków dynamicznych", a więc takich, które składają się z wielu różnych obrazów. Może to być przydatne np. podczas wizualizacji kolejnych kroków danego algorytmu. Poszczególne klatki określane są jako obiekty `Scene`, które przyjmują listę `PointsCollection` oraz listę `LinesCollection` w swoim konstruktorze. Obiekt `Plot` przyjmuje w parametrze konstruktora `scenes` właśnie listę takich scen.

Na rysunku przyciski `Następny` i `Poprzedni` służą do nawigacji pomiędzy nimi.

```python
scenes=[Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)]), 
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])], "Scene 1"), 
        Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)], color='red'), 
               PointsCollection([(5, -2), (2, 2), (-2, 1)], color='black')], 
              [LinesCollection([[(-1,2),(-2,3)], [(0,-1),(-1,0)]])],"Scene 2")]

plot=Plot(scenes = scenes)
plot.draw()
```

##### Zapis i odczyt z pliku

Klasa Plot posiada metodę `toJson()`, która zwraca string zawierający listę scen w formacie JSON. Taki string można normalnie zapisać do pliku stosując normalne sposoby dostępne w Pythonie. Wczytanie listy scen z pliku dokonuje się poprzez podanie parametru `json` w kostruktorze `Plot`.

```python
scenes=[Scene([PointsCollection([(-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])]), 
        Scene([PointsCollection([(1, 2), (3, 1.5)], color='red'), 
               PointsCollection([(5, -2)], color='black')])]
plot = Plot(scenes)

with open('somefile.json', 'w') as file:
    file.write(plot.toJson())
    
#somefile.json: [{"points": [[[-2, -1]]], "lines": [[[[1, 2], [2, 3]], [[0, 1], [1, 0]]]]}, {"points": [[[1.0, 2.0], [3.0, 1.5]], [[5, -2]]], "lines": []}]
    
with open('somefile.json', 'r') as file:
    json = file.read()
    
plot2 = Plot(json=json)
plot2.draw()
```

##### Operowanie na dodanych punktach

Możemy bardzo łatwo rysować nowe punkty i linie. Aby je później wykorzystać sugerowanym sposobem jest utworzenie nowej komórki, w której utworzymy nowy `Plot` i skorzystamy z metod `get_added_points()` i `get_added_points()` lub `get_added_elements()` (zwraca scenę) z pierwszego `Plot`u.

```python 
#Dodaj tu punkty, figury i/lub odcinki!
plot1 = Plot(points=[PointsCollection([(-2,-1),(5,3)], color='red')])
plot1.draw()

plot2 = Plot([plot1.get_added_elements()])
plot2.draw()
```

## Pomocnicze funkcje oraz biblioteki

In [ ]:
from sortedcontainers import SortedSet

def point_position(a: tuple[float,float], b:tuple[float,float], c:tuple[float,float], epsilon=1e-10) -> int:
    """Find position of a point c relative to segment ab using 3x3 matrix determinant. 

    :param tuple[float,float] a: First point of a segment/vector
    :param tuple[float,float] b: Second point of a segment/vector
    :param tuple[float,float] c: Point which is considered
    "param float epsilon: Tolerance for collinearity
    :return int: 1 if point lies on the left, 0 if is collinear, -1 if is on the right side of a segment ab.
    """

    # Calculate matrix
    matrix = [[],[],[]]
    matrix[0] = [a[0], a[1], 1]
    matrix[1] = [b[0], b[1], 1]
    matrix[2] = [c[0], c[1], 1]

    f1 = matrix[0][0] * matrix[1][1] * matrix[2][2]
    f2 = matrix[0][1] * matrix[1][2] * matrix[2][0]
    f3 = matrix[1][0] * matrix[2][1] * matrix[0][2]
    f4 = matrix[0][2] * matrix[1][1] * matrix[2][0]
    f5 = matrix[1][2] * matrix[2][1] * matrix[0][0]
    f6 = matrix[1][0] * matrix[0][1] * matrix[2][2]
    
    determinant = f1+f2+f3-f4-f5-f6

    # Return relative position

    # Point c is on the left side of segment ab
    if determinant > epsilon:
        return 1
    # Point c is collinear with segment ab
    elif -epsilon <= determinant <= epsilon:
        return 0
    # Point c is on the right side of segment ab
    else:
        return -1

def do_two_segment_intersect(segment1: tuple[tuple[float,float],tuple[float,float]], segment2: tuple[tuple[float,float], tuple[float,float]]) -> bool:
    """Algorithm for checking whether two segment intersect. 
    General case: For first segment: second segment has to have it's points on different sides of first segment (one on the left and one on the right), same condition must fulfill second segment, relative to fist segment. 
    Special case: When segments are collinear, then we have to check their x and y projections on axis.

    :param tuple[tuple[float,float],tuple[float,float]] segment1: First segment
    :param tuple[tuple[float,float], tuple[float,float]] segment_2: Second segment
    :return bool: True, if two segments intersect
    """
  
    # First segment
    first_segment_result = [point_position(segment1[0], segment1[1], segment2[0]),point_position(segment1[0], segment1[1], segment2[1])]
    # Sort results
    if first_segment_result[0] > first_segment_result[1]:
        first_segment_result = first_segment_result[::-1]

    # Second segment 
    second_segment_result = [point_position(segment2[0], segment2[1], segment1[0]),point_position(segment2[0], segment2[1], segment1[1])]
    # Sort results
    if second_segment_result[0] > second_segment_result[1]:
        second_segment_result = second_segment_result[::-1]

    # General case
    if first_segment_result == [-1, 1] and second_segment_result == [-1,1]:
        return True

    # Special case
    if first_segment_result == [0,0] and second_segment_result == [0,0]:
        # Check x projection
        max_x_segment1 = max(segment1, key=lambda x: x[0])[0]
        min_x_segment1 = min(segment1, key=lambda x: x[0])[0]
        min_x_segment2 = min(segment2, key=lambda x: x[0])[0]
        max_x_segment2 = max(segment2, key=lambda x: x[0])[0]
        if max_x_segment1 < min_x_segment2 or max_x_segment2 < min_x_segment1:
            return False
        
        # Check y projection
        max_y_segment1 = max(segment1, key=lambda x: x[1])[1]
        min_y_segment1 = min(segment1, key=lambda x: x[1])[1]
        min_y_segment2 = min(segment2, key=lambda x: x[1])[1]
        max_y_segment2 = max(segment2, key=lambda x: x[1])[1]
        if max_y_segment1 < min_y_segment2 or max_y_segment2 < min_y_segment1:
            return False
        
        return True

    return False

def get_intersection_point(line1: tuple[tuple[float,float]], line2: tuple[tuple[float,float]], epsilon: float =1e-14) -> tuple[float, float]:
    """Calculate intersection point of line p1p2 and segment p3p4 with formula from: 
        https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection

    :param tuple[tuple[float,float]] line1: First line
    :param tuple[tuple[float,float]] line2: Second line
    :param float epsilon: Add epsilon to point comparison (helpful when segment has the same x coordinates)
    :return tuple[float, float]: Intersection point
    """

    p1,p2 = line1
    p3,p4 = line2

    denominator = (p1[0] - p2[0])*(p3[1] - p4[1]) - \
        (p1[1] - p2[1]) * (p3[0] - p4[0])

    if denominator == 0:
        return (np.inf, np.inf)

    x_numerator = (p1[0]*p2[1] - p1[1]*p2[0])*(p3[0]-p4[0]) - \
        (p1[0] - p2[0])*(p3[0]*p4[1] - p3[1]*p4[0])
    y_numerator = (p1[0]*p2[1] - p1[1]*p2[0])*(p3[1]-p4[1]) - \
        (p1[1] - p2[1])*(p3[0]*p4[1] - p3[1]*p4[0])

    intersection_point = (x_numerator/denominator, y_numerator/denominator)

    if p3[0] < p4[0]:
        segment_boundary_min_x = p3[0]
        segment_boundary_max_x = p4[0]
    else:
        segment_boundary_min_x = p4[0]
        segment_boundary_max_x = p3[0]

    if p3[1] < p4[1]:
        segment_boundary_min_y = p3[1]
        segment_boundary_max_y = p4[1]
    else:
        segment_boundary_min_y = p4[1]
        segment_boundary_max_y = p3[1]

    if segment_boundary_min_x - epsilon <= intersection_point[0] <= segment_boundary_max_x + epsilon and segment_boundary_min_y - epsilon <= intersection_point[1] <= segment_boundary_max_y + epsilon:
        return intersection_point

    return (np.inf, np.inf)

## Losowanie odcinków z podanego zakresu współrzędnych $[a,b)$

In [ ]:
def generate_lines(n=20, xrange=[0,10], yrange=[0,10]):
    lines = []
    x_coords = set()
    for _ in range(n):
        while True:
            p1 = tuple(np.random.uniform(xrange[0], xrange[1], size=2))
            p2 = tuple(np.random.uniform(yrange[0], yrange[1], size=2))

            if not (p1[0] == p2[0] or p1[0] in x_coords or p2 in x_coords):
                break
        
        x_coords.add(p1[0])
        x_coords.add(p2[0])
        line = (p1,p2)
        line = sorted(line, key=lambda x: x[0])

        lines.append(line)
    
    return lines

## Odczyt i zapis krawędzi do pliku JSON

In [ ]:
def read_lines(filename="lines") -> list[tuple[float,float], tuple[float,float]]:
    try:
        with open(f"{filename}.json", 'r') as file:
            return js.load(file)['lines']
    except FileNotFoundError:
        print("File not found!")
        return []
def save_lines(filename="lines", lines: list[tuple[float,float], tuple[float,float]] = []):
    if len(lines) == 0:
        print("No lines to save!")
        return
    
    with open(f"{filename}.json", 'w') as file:
        js.dump({"lines": lines}, file)


## Algorytm zamiatania sprawdzający,  czy  choć  jedna  para  odcinków w zadanym zbiorze się przecina

In [ ]:
class Line:
    def __init__(self, p1,p2) -> None:
        self.p1 = p1
        self.p2 = p2
        self.a = (p2[1] - p1[1]) / (p2[0] - p1[0])
        self.b = p2[1] - self.a * p2[0]

    def key(self):
        return ((self.p1[0],self.p1[1]), (self.p2[0], self.p2[1]))

    def __eq__(self, other):
        return isinstance(other, Line) and self.p1 == other.p1 and self.p2 == other.p2

    def __hash__(self):
        return hash(self.key())

    def toStructureT(self):
        return (self.a,self.b,(self.p1,self.p2))

In [ ]:
def does_any_pair_of_lines_intersect(lines, print_information=True):
    # Initialize Q
    Q = SortedSet(key=lambda x: x[0])
    event_dict = {}

    min_y = float('inf')
    max_y = float('-inf')

    for line in lines:
        line.sort(key=lambda x: x[0])
        p1,p2 = line
        p1 = tuple(p1)
        p2 = tuple(p2)

        min_y = min(min_y, p1[1], p2[1])
        max_y = max(max_y, p1[1], p2[1])

        line_obj = Line(p1, p2)      

        Q.add(p1)
        event_dict[p1] = ("left", line_obj, None)
        Q.add(p2)
        event_dict[p2] = ("right", line_obj, None)


    # Initialize T
    T = SortedSet()

    # Save intersections
    intersections = set()
    intersections_with_lines = set()
    detected_points = set()

    # Scene container
    scenes = []
    sweep_line = [(Q[0][0], min_y), (Q[0][0], max_y)]

    while Q:
        px = Q.pop(0)  
        placement ,line, line2 = event_dict[px]
        event_point = (px[0],px[1])

        sweep_line[0] = (px[0], min_y)
        sweep_line[1] = (px[0], max_y)

        px = px[0]

        # Update T key
        T_update = SortedSet(key=lambda x: x.a * px + x.b)
        T_update.update(T)
        T = T_update

   
        # If p is the left point of line
        if placement == "left":
            T.add(line)

            # Check if there exists intersection with neighbour
            if T.index(line) - 1 >= 0 and do_two_segment_intersect(T[T.index(line)-1].key(), line.key()):
                # Get intersection point
                intersection_point = get_intersection_point(T[T.index(line)-1].key(), line.key())

                scenes.insert(0, Scene(title="Znaleziono przecięcie się odcinków", lines=[LinesCollection(lines)], points=[PointsCollection([intersection_point],color="red"), PointsCollection([event_point], color="green")]))

                if print_information: print(f"Znaleziono przecięcie się odcinków w punkcie: {intersection_point} (gdy analizowano punkt: {event_point})")


                return True, scenes
            
            if T.index(line) + 1 < len(T) and do_two_segment_intersect(T[T.index(line)+1].key(), line.key()):
                # Get intersection point
                intersection_point = get_intersection_point(T[T.index(line)+1].key(), line.key())

                scenes.insert(0, Scene(title="Znaleziono przecięcie się odcinków", lines=[LinesCollection(lines)], points=[PointsCollection([intersection_point],color="red"), PointsCollection([event_point], color="green")]))

                if print_information: print(f"Znaleziono przecięcie się odcinków w punkcie: {intersection_point} (gdy analizowano punkt: {event_point})")

                return True, scenes


        # If p is the right point of line
        elif placement == "right":
            if T.index(line) - 1 >= 0 and T.index(line) + 1 < len(T) and do_two_segment_intersect(T[T.index(line)-1].key(), T[T.index(line)+1].key()):
                intersection_point = get_intersection_point(T[T.index(line)+1].key() ,T[T.index(line)-1].key())

                scenes.insert(0, Scene(title="Znaleziono przecięcie się odcinków", lines=[LinesCollection(lines)], points=[PointsCollection([intersection_point],color="red"), PointsCollection([event_point], color="green")]))

                if print_information: print(f"Znaleziono przecięcie się odcinków w punkcie: {intersection_point} (gdy analizowano punkt: {event_point})")

                return True, scenes


            T.remove(line)

        scenes.append(Scene(title="Zamiatanie w toku...", lines=[LinesCollection(lines),LinesCollection(list(map(lambda x: x.key() ,T)), color="green"),LinesCollection([sweep_line[:]], color="red")], points=[PointsCollection(list(detected_points), color="red"), PointsCollection([event_point], color="orange")]))

    scenes[-1].title="\nKoniec zamiatania :)"
    scenes.insert(0,Scene(title="Wykryte punkty przecięć", lines=[LinesCollection(lines)], points=[PointsCollection(list(detected_points), color="red")]))

    if print_information:
        print("============== Punkty przecięć ================")
        print(f"Liczba przecięć: {len(intersections_with_lines)}")
        for point, line1, line2 in intersections_with_lines:
            print(f"Punkt: {point}, dla odcinków: {line1} oraz {line2}")
    
    return intersections_with_lines, scenes


## Algorytm wyznaczający wszystkie przecięcia  odcinków

In [ ]:
def get_lines_intersection_points(lines, print_intersection_points=True):
    # Initialize Q
    Q = SortedSet(key=lambda x: x[0])
    event_dict = {}

    min_y = float('inf')
    max_y = float('-inf')

    for line in lines:
        line.sort(key=lambda x: x[0])
        p1,p2 = line
        p1 = tuple(p1)
        p2 = tuple(p2)

        min_y = min(min_y, p1[1], p2[1])
        max_y = max(max_y, p1[1], p2[1])

        line_obj = Line(p1, p2)      

        Q.add(p1)
        event_dict[p1] = ("left", line_obj, None)
        Q.add(p2)
        event_dict[p2] = ("right", line_obj, None)


    # Initialize T
    T = SortedSet()

    # Save intersections
    intersections = set()
    intersections_with_lines = set()
    detected_points = set()

    # Scene container
    scenes = []
    sweep_line = [(Q[0][0], min_y), (Q[0][0], max_y)]

    while Q:
        px = Q.pop(0)  
        placement ,line, line2 = event_dict[px]
        event_point = (px[0],px[1])

        sweep_line[0] = (px[0], min_y)
        sweep_line[1] = (px[0], max_y)

        px = px[0]

        # Update T key
        T_update = SortedSet(key=lambda x: x.a * px + x.b)
        T_update.update(T)
        T = T_update

   
        # If p is the left point of line
        if placement == "left":
            T.add(line)

            # Check if there exists intersection with neighbour
            if T.index(line) - 1 >= 0 and do_two_segment_intersect(T[T.index(line)-1].key(), line.key()):
                # Get intersection point
                segment_intersection_point = get_intersection_point(T[T.index(line)-1].key(), line.key())

                if segment_intersection_point not in Q and segment_intersection_point not in intersections and segment_intersection_point != (np.inf, np.inf) and segment_intersection_point[0] >= px:
                    Q.add(segment_intersection_point)
                    event_dict[segment_intersection_point] = ( "intersection", line, T[T.index(line)-1])
                    detected_points.add(segment_intersection_point)

            
            if T.index(line) + 1 < len(T) and do_two_segment_intersect(T[T.index(line)+1].key(), line.key()):
                # Get intersection point
                segment_intersection_point = get_intersection_point(T[T.index(line)+1].key(), line.key())

                if segment_intersection_point not in Q and segment_intersection_point not in intersections and segment_intersection_point != (np.inf, np.inf) and segment_intersection_point[0] >= px:
                    Q.add(segment_intersection_point)
                    event_dict[segment_intersection_point] = ( "intersection", line, T[T.index(line)+1])
                    detected_points.add(segment_intersection_point)


        # If p is the right point of line
        elif placement == "right":
            if T.index(line) - 1 >= 0 and T.index(line) + 1 < len(T) and do_two_segment_intersect(T[T.index(line)-1].key(), T[T.index(line)+1].key()):
                segment_intersection_point = get_intersection_point(T[T.index(line)+1].key() ,T[T.index(line)-1].key())

                if not segment_intersection_point in intersections and segment_intersection_point != (np.inf, np.inf) and segment_intersection_point[0] >= px:
                    Q.add(segment_intersection_point)
                    event_dict[segment_intersection_point] = ( "intersection",T[T.index(line)-1], T[T.index(line)+1])
                    detected_points.add(segment_intersection_point)


            T.remove(line)

        # Intersection point
        elif placement == "intersection":
            # if event_point not in intersections:
            intersections_with_lines.add((event_point,line.key(),line2.key()))
            intersections.add(event_point)

            # Reverse lines
            T.remove(line)
            T.remove(line2)
        

            T_update = SortedSet(key=lambda x: x.a * (px + 1e-10) + x.b)
            T_update.update(T)
            T = T_update

            T.add(line)
            T.add(line2)
            

            if T.index(line) - 1 >= 0 and do_two_segment_intersect(T[T.index(line)-1].key(), line.key()):
                # Get intersection point
                segment_intersection_point = get_intersection_point(line.key() ,T[T.index(line)-1].key())
            
                if segment_intersection_point not in Q and segment_intersection_point not in intersections and segment_intersection_point != (np.inf, np.inf) and segment_intersection_point[0] >= px:   
                    Q.add(segment_intersection_point)
                    event_dict[segment_intersection_point] = ( "intersection", line, T[T.index(line)-1])
                    detected_points.add(segment_intersection_point)

            if T.index(line) + 1 < len(T) and do_two_segment_intersect(T[T.index(line)+1].key(), line.key()):
                # Get intersection point
                segment_intersection_point = get_intersection_point(line.key() ,T[T.index(line)+1].key())

                if segment_intersection_point not in Q and segment_intersection_point not in intersections and segment_intersection_point != (np.inf, np.inf) and segment_intersection_point[0] >= px:   
                    Q.add(segment_intersection_point)
                    event_dict[segment_intersection_point] = ( "intersection", line, T[T.index(line)+1])
                    detected_points.add(segment_intersection_point)

            if T.index(line2) - 1 >= 0 and do_two_segment_intersect(T[T.index(line2)-1].key(), line2.key()):
                # Get intersection point
                segment_intersection_point = get_intersection_point(line2.key() ,T[T.index(line2)-1].key())

                if segment_intersection_point not in Q and segment_intersection_point not in intersections and segment_intersection_point != (np.inf, np.inf) and segment_intersection_point[0] >= px:   
                    Q.add(segment_intersection_point)
                    event_dict[segment_intersection_point] = ( "intersection", line2, T[T.index(line2)-1])
                    detected_points.add(segment_intersection_point)

            if T.index(line2) + 1 < len(T) and do_two_segment_intersect(T[T.index(line2)+1].key(), line2.key()):
                # Get intersection point
                segment_intersection_point = get_intersection_point(line2.key() ,T[T.index(line2)+1].key())

                if segment_intersection_point not in Q and segment_intersection_point not in intersections and segment_intersection_point != (np.inf, np.inf) and segment_intersection_point[0] >= px:   
                    Q.add(segment_intersection_point)
                    event_dict[segment_intersection_point] = ( "intersection", line2, T[T.index(line2)+1])
                    detected_points.add(segment_intersection_point)
       
 
        scenes.append(Scene(title="Zamiatanie w toku...", lines=[LinesCollection(lines),LinesCollection(list(map(lambda x: x.key() ,T)), color="green"),LinesCollection([sweep_line[:]], color="red")], points=[PointsCollection(list(detected_points), color="red"), PointsCollection([event_point], color="orange")]))

    scenes[-1].title="\nKoniec zamiatania :)"
    scenes.insert(0,Scene(title="Wykryte punkty przecięć", lines=[LinesCollection(lines)], points=[PointsCollection(list(detected_points), color="red")]))

    if print_intersection_points:
        print("============== Punkty przecięć ================")
        print(f"Liczba przecięć: {len(intersections_with_lines)}")
        for point, line1, line2 in intersections_with_lines:
            print(f"Punkt: {point}, dla odcinków: {line1} oraz {line2}")
    
    return intersections_with_lines, scenes


## Generacja różnych zbiorów odcinków

In [ ]:
lines = read_lines("line_set_1")
if lines == []:
    # Generate random segments
    save_lines("line_set_1",generate_lines(10))
    lines = read_lines("line_set_1")

plot1 = Plot(title="line_set_1", lines=[LinesCollection(lines)])
plot1.draw()

In [ ]:
lines = read_lines("line_set_2")
if lines == []:
    # Generate random segments
    save_lines("line_set_2",generate_lines())
    lines = read_lines("line_set_2")

plot2 = Plot(title="line_set_2", lines=[LinesCollection(lines)])
plot2.draw()

In [ ]:
lines = read_lines("line_set_3")
if lines == []:
    # Generate random segments
    save_lines("line_set_3",generate_lines())
    lines = read_lines("line_set_3")

plot3 = Plot(title="line_set_3", lines=[LinesCollection(lines)])
plot3.draw()

In [ ]:
lines = read_lines("line_set_4")
if lines == []:
    # Generate random segments
    save_lines("line_set_4",generate_lines())
    lines = read_lines("line_set_4")

plot4 = Plot(title="line_set_4", lines=[LinesCollection(lines)])
plot4.draw()

In [ ]:
lines = read_lines("line_set_5")
if lines != []:
    plot5 = Plot(title="line_set_5", xlim=[0,10], ylim=[0,10], lines=[LinesCollection(lines)])
    plot5.draw()
else:
    plot5 = Plot(title="line_set_5", xlim=[0,10], ylim=[0,10])
    plot5.draw()

In [ ]:
# Use this code to save lines above
if plot5.get_added_elements():
    save_lines("line_set_5", plot5.get_added_lines()[0].lines)
    lines = read_lines("line_set_5")
    plot5 = Plot(title="line_set_5", lines=[LinesCollection(lines)])


In [ ]:
lines = read_lines("line_set_6")
if lines != []:
    plot6 = Plot(title="line_set_6", xlim=[0,10], ylim=[0,10], lines=[LinesCollection(lines)])
    plot6.draw()
else:
    plot6 = Plot(title="line_set_6", xlim=[0,10], ylim=[0,10])
    plot6.draw()

In [ ]:
# Use this code to save lines above
if plot6.get_added_elements():
    save_lines("line_set_6", plot6.get_added_lines()[0].lines)
    lines = read_lines("line_set_6")
    plot6 = Plot(title="line_set_6", lines=[LinesCollection(lines)])


In [ ]:
lines = read_lines("line_set_7")
if lines != []:
    plot7 = Plot(title="line_set_7", xlim=[0,10], ylim=[0,10], lines=[LinesCollection(lines)])
    plot7.draw()
else:
    plot7 = Plot(title="line_set_7", xlim=[0,10], ylim=[0,10])
    plot7.draw()

In [ ]:
# Use this code to save lines above
if plot7.get_added_elements():
    save_lines("line_set_7", plot7.get_added_lines()[0].lines)
    lines = read_lines("line_set_7")
    plot7 = Plot(title="line_set_7", lines=[LinesCollection(lines)])


## Testowanie algorytmu sprawdzającego, czy  choć  jedna  para  odcinków w zadanym zbiorze się przecina 

In [ ]:
considered_plot = plot1
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
has_intersection, scenes = does_any_pair_of_lines_intersect(plot_lines)
plot = Plot(title="line_set_1_has_intersection", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot2
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
has_intersection, scenes = does_any_pair_of_lines_intersect(plot_lines)
plot = Plot(title="line_set_2_has_intersection", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot3
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
has_intersection, scenes = does_any_pair_of_lines_intersect(plot_lines)
plot = Plot(title="line_set_3_has_intersection", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot4
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
has_intersection, scenes = does_any_pair_of_lines_intersect(plot_lines)
plot = Plot(title="line_set_4_has_intersection", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot5
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
has_intersection, scenes = does_any_pair_of_lines_intersect(plot_lines)
plot = Plot(title="line_set_5_has_intersection", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot6
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
has_intersection, scenes = does_any_pair_of_lines_intersect(plot_lines)
plot = Plot(title="line_set_6_has_intersection", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot7
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
has_intersection, scenes = does_any_pair_of_lines_intersect(plot_lines)
plot = Plot(title="line_set_7_has_intersection", scenes=scenes)
plot.draw()

## Testowanie algorytmu wyznaczającego wszystkie przecięcia  odcinków

In [ ]:
considered_plot = plot1
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
intersections, scenes = get_lines_intersection_points(plot_lines)
plot = Plot(title="line_set_1_intersection_points", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot2
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
intersections, scenes = get_lines_intersection_points(plot_lines)
plot = Plot(title="line_set_2_intersection_points", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot3
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
intersections, scenes = get_lines_intersection_points(plot_lines)
plot = Plot(title="line_set_3_intersection_points", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot4
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
intersections, scenes = get_lines_intersection_points(plot_lines)
plot = Plot(title="line_set_4_intersection_points", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot5
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
intersections, scenes = get_lines_intersection_points(plot_lines)
plot = Plot(title="line_set_5_intersection_points", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot6
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
intersections, scenes = get_lines_intersection_points(plot_lines)
plot = Plot(title="line_set_6_intersection_points", scenes=scenes)
plot.draw()

In [ ]:
considered_plot = plot7
plot_lines = []
if len(considered_plot.scenes) > 0 and len(considered_plot.scenes[0].lines) > 0:
    plot_lines.extend(considered_plot.scenes[0].lines[0].lines)
if len(considered_plot.get_added_lines()) > 0:
    plot_lines.extend(considered_plot.get_added_lines()[0].lines)
intersections, scenes = get_lines_intersection_points(plot_lines)
plot = Plot(title="line_set_7_intersection_points", scenes=scenes)
plot.draw()